# Preconditioner Series: Sparse Approximate Inverse

The premise of sparse approximate inverse (SAI) preconditioners is to explicitly calculate a sparse approximation of $A^{-1}$ such that $M\approx A^{-1}$.

Below, I implement the following SAI methods with the help of PyTorch:
1. Frobenius norm minimization
    1. SPAI algorithm
    2. MR algorithm
    3. Self-preconditioned MR algorithm
2. Factorized sparse approximate inverses
    1. Biconjugation algorithm
3. Inverse ILU techniques

## Imports

In [1]:
import numpy as np
import torch
# import torch.cuda as tc
from torch.autograd import Variable

np.set_printoptions(suppress=True)

%load_ext autoreload
%autoreload 2

## Helper methods

In [2]:
def sparsity(M):
    return 1 - np.count_nonzero(M.numpy()) / torch.numel(M)

def report(A, M):
    print((torch.eye(n) - torch.mm(A, M)).norm(2).item(), sparsity(M))

## Frobenius norm minimization

This class of methods is based on the computation of a sparse matrix $M=A^{-1}$ from the following constrained minimization problem:

$$ \min_{M\in S} \lVert I-AM\rVert_F$$

where $S$ is a set of sparse matrices. The above problem can be solved for a right approximate inverse. The left approxiate inverse can be calculated by minimizing $\lVert I-MA\rVert_F$.

### SPAI algorithm

One of the most successful algorithms proposed in this class is known as the SPAI preconditioner. The algorithm is described [here](https://arxiv.org/abs/1503.04500) and [here](http://www.mathcs.emory.edu/~benzi/Web_papers/comp.pdf) and is as follows:

For every column $m_j$ of $M$:

1. Choose an initial sparsity pattern $J$
2. 

In [ ]:
def spai(A):
    
    
    return M

### MR algorithm

The [MR algorithm](https://www.cc.gatech.edu/~echow/pubs/newapinv.pdf)

For an n-by-n matrix $A$:

1. Choose an initial guess $M=M_0=[m_1, m_2,\dots,m_n]$
2. For each column $m_j$, $j=1,2,\dots,n$
    1. For $i$ in $1,2,\dots,n_i$
        1. $r_j=e_j-Am_j$
        2. $\alpha_j=r_j^TAr_j/((Ar_j)^T(Ar_j))$
        3. $m_j=m_j+\alpha_jr_j$
        4. Numerical dropping on $m_j$
        
**Numerical dropping?**


In [94]:
def drop(m):
    

def mr(A, M, ni=10, tol=1e-3):
    n = A.shape[0]
    e = np.eye(n)
    
    for j in range(n):
        for i in range(ni):
            r = e[j] - A @ M[:,j]
            a = r.T @ A @ r / ((A @ r).T @ (A @ r))
            M[:,j] += a * r
            drop(M[:,j])

SyntaxError: unexpected EOF while parsing (<ipython-input-94-b9c1c93cfe2e>, line 1)

In [5]:
n = 10
A = np.random.rand(n, n)
M0 = np.eye(n)

# mr(A, M0)
M0

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

### Self-preconditioned MR algorithm

In [ ]:
def self_mr(A, M0, ni, tol)

## Factorized sparse approximate inverses

### Biconjugation algorithm

## Inverse ILU techniques

## Plain old L1-regularization

Having learned about L1-regularization in machine learning, I wondered how such a simple method would do when calculating a sparse approximate inverse.

The minimization problem here is:

$$ \min_{M} \lVert I-AM\rVert_F + \alpha\lVert M\rVert_1$$

In [5]:
def loss(A, M, n, alpha):
    return torch.norm(torch.mm(A, M) - torch.eye(n), p='fro') + alpha * torch.norm(M ,p=1)

def plain_l1(A, lr=1e-3, alpha=1e5, eps=1e-3, rtol=1e-5, max_iter=10000):
    
    n = A.shape[0]
    pA = Variable(torch.FloatTensor(A), requires_grad=False)
    pM = Variable(torch.randn(n, n), requires_grad=True)
    
    opt = torch.optim.SGD([pM], lr)
    
    for i in range(max_iter):
        opt.zero_grad()
        pA[torch.abs(pA) < eps] = 0
        l = loss(pA, pM, n, alpha)
        l.backward()
        opt.step()
        
        if i % 1000 == 999:
            report(pA.data, pM.data)
    
    return pM

In [6]:
n = 10
A = np.random.rand(n, n)
M = plain_l1(A, lr=1e-3, alpha=1, eps=1e-3)

M.data.numpy()

4.953068733215332 0.0
3.1871531009674072 0.0
3.158658027648926 0.0
3.159771680831909 0.0
3.1595215797424316 0.0
3.159792900085449 0.0
3.1589648723602295 0.0
3.160285472869873 0.0
3.159102201461792 0.0
3.1588714122772217 0.0


array([[ 0.00045629, -0.00012942, -0.0002652 ,  0.00078636,  0.00121963,
         0.00010997,  0.00093774,  0.00093873, -0.00032723,  0.00013044],
       [ 0.00005528,  0.00081641,  0.00032489,  0.00007263,  0.00080921,
        -0.00013259, -0.00047741, -0.00032341,  0.00071864,  0.00011489],
       [ 0.00059126,  0.00071992, -0.00067428, -0.00064733,  0.00070266,
        -0.00001875,  0.00068053, -0.00034219,  0.00083424, -0.0003781 ],
       [ 0.00065093,  0.00109324, -0.00061818,  0.00081611, -0.00016427,
         0.00051789,  0.0004282 , -0.00017426, -0.00082078,  0.00005166],
       [ 0.00020285, -0.00079511, -0.00095196,  0.00124755,  0.00062162,
         0.00095477, -0.00011105,  0.00047384, -0.00035185, -0.00055214],
       [-0.00029456,  0.00030927,  0.00089606, -0.0008059 , -0.00013875,
         0.0011206 ,  0.00026234,  0.00013471,  0.00089078,  0.00085093],
       [-0.00022143,  0.00032813,  0.00033733,  0.00001479,  0.00075345,
         0.00014675,  0.00091832,  0.00000916

## Resources

[A comparative study of sparse approximate inverse preconditioners](http://www.mathcs.emory.edu/~benzi/Web_papers/comp.pdf)

[A Residual Based Sparse Approximate Inverse Preconditioning Procedure for Large Sparse Linear Systems](https://arxiv.org/abs/1503.04500)